## LBO Model - Sample Case Study

### I. Data Preparation

In [80]:
# Import packages 
import pandas as pd
import numpy as np

### II. Calculate the Purchase Price of XXX Company

In [81]:
# Create a dict to save variables for purchase price assumptions
entry_assumptions = {
    'purchase_multiple':10,
    'LTM_Rev':150,
    'EBITDA_Margin':.3,
    'debt_ratio': .4,
    'equity_ratio': .6,
}
entry_assumptions['LTM_EBITDA'] = entry_assumptions['LTM_Rev'] * entry_assumptions['EBITDA_Margin']
entry_assumptions['purchase_TEV'] = entry_assumptions['purchase_multiple'] * entry_assumptions['LTM_EBITDA']
entry_assumptions

{'purchase_multiple': 10,
 'LTM_Rev': 150,
 'EBITDA_Margin': 0.3,
 'debt_ratio': 0.4,
 'equity_ratio': 0.6,
 'LTM_EBITDA': 45.0,
 'purchase_TEV': 450.0}

### III. Create the Sources & Uses Schedules

#### Uses Schedule

In [82]:
# MOD - Create the uses schedule 
uses_sche = {
    'fin_adv_fees':.02,
}
uses_sche['fees'] = uses_sche['fin_adv_fees'] * entry_assumptions['purchase_TEV']
uses_sche['total_uses'] = uses_sche['fees'] + entry_assumptions['purchase_TEV']
uses_sche

{'fin_adv_fees': 0.02, 'fees': 9.0, 'total_uses': 459.0}

#### Sources Schedule

In [83]:
# MOD - Create the sources schedule 
sources_sche = {
    'senior_bank':3,
    'sellers_note':1,
}
sources_sche['sen_bank_debt'] = sources_sche['senior_bank'] * entry_assumptions['LTM_EBITDA']
sources_sche['sellers_notes'] = sources_sche['sellers_note'] * entry_assumptions['LTM_EBITDA']
sources_sche['equity_funding'] = uses_sche['total_uses'] - sources_sche['sellers_notes'] - sources_sche['sen_bank_debt']
sources_sche['total_sources'] = sources_sche['sen_bank_debt'] + sources_sche['sellers_notes'] + sources_sche['equity_funding']
sources_sche

{'senior_bank': 3,
 'sellers_note': 1,
 'sen_bank_debt': 135.0,
 'sellers_notes': 45.0,
 'equity_funding': 279.0,
 'total_sources': 459.0}

#### Equity Split 

In [84]:
# Create the equity split table 
equity_split = {
    'mgmt_equity_contri':.1,
    'mgmt_earnout1':.15,
    'mgmt_earnout2':.25,
}
equity_split['mgmt_equity'] = sources_sche['equity_funding'] * equity_split['mgmt_equity_contri']
equity_split['shareholder_loan'] = entry_assumptions['LTM_EBITDA'] * 1
equity_split['sponsor_equity'] = sources_sche['equity_funding'] - (equity_split['mgmt_equity'] + equity_split['shareholder_loan']) 
equity_split

{'mgmt_equity_contri': 0.1,
 'mgmt_earnout1': 0.15,
 'mgmt_earnout2': 0.25,
 'mgmt_equity': 27.900000000000002,
 'shareholder_loan': 45.0,
 'sponsor_equity': 206.1}

### IV. Income Statement

In [85]:
# Create a dataframe to take in 5 years of projection 
years = [1, 2, 3, 4, 5, 6]
rows = ['Sales', 'EBITDA', 'less: D&A', 'EBIT', 'less: Interest', 'Pre-Tax Income', 'less: Taxes', 'Net Income']
IS = pd.DataFrame(data = None,index = rows, columns = years)
IS

,1,2,3,4,5,6
Sales,NaN,NaN,NaN,NaN,NaN,NaN
EBITDA,NaN,NaN,NaN,NaN,NaN,NaN
less: D&A,NaN,NaN,NaN,NaN,NaN,NaN
EBIT,NaN,NaN,NaN,NaN,NaN,NaN
less: Interest,NaN,NaN,NaN,NaN,NaN,NaN
Pre-Tax Income,NaN,NaN,NaN,NaN,NaN,NaN
less: Taxes,NaN,NaN,NaN,NaN,NaN,NaN
Net Income,NaN,NaN,NaN,NaN,NaN,NaN


In [86]:
# Create a dictionary to save variables for Income Statement projection
IS_assumptions = {
    "rev_growth": 1.1,
    "int_rate": .13,
    "EBITDA_margin": entry_assumptions['EBITDA_Margin'],
    'tax_rate':.3,
    'CAPEX % of Sales':.07,
    'Depre % of Sales':0.05,
    'change_in_NWC':3,
}
IS_assumptions

{'rev_growth': 1.1,
 'int_rate': 0.13,
 'EBITDA_margin': 0.3,
 'tax_rate': 0.3,
 'CAPEX % of Sales': 0.07,
 'Depre % of Sales': 0.05,
 'change_in_NWC': 3}

In [87]:
# Assume a flat 10% growth rate for the next 5 years 
IS[1].loc['Sales'] = entry_assumptions['LTM_Rev']
i = 2
while i <= len(IS.loc['Sales']):
    IS[i].loc['Sales'] = IS[(i-1)].loc['Sales'] * 1.10
    i += 1
    
IS

,1,2,3,4,5,6
Sales,150,165.0,181.5,199.65,219.615,241.5765
EBITDA,NaN,NaN,NaN,NaN,NaN,NaN
less: D&A,NaN,NaN,NaN,NaN,NaN,NaN
EBIT,NaN,NaN,NaN,NaN,NaN,NaN
less: Interest,NaN,NaN,NaN,NaN,NaN,NaN
Pre-Tax Income,NaN,NaN,NaN,NaN,NaN,NaN
less: Taxes,NaN,NaN,NaN,NaN,NaN,NaN
Net Income,NaN,NaN,NaN,NaN,NaN,NaN


In [88]:
# Project EBITDA using a constant % EBITDA margin assumptions
IS.loc['EBITDA'] = IS.loc['Sales'] * IS_assumptions['EBITDA_margin']
IS

,1,2,3,4,5,6
Sales,150,165.0,181.5,199.65,219.615,241.5765
EBITDA,45.0,49.5,54.45,59.895,65.8845,72.47295
less: D&A,NaN,NaN,NaN,NaN,NaN,NaN
EBIT,NaN,NaN,NaN,NaN,NaN,NaN
less: Interest,NaN,NaN,NaN,NaN,NaN,NaN
Pre-Tax Income,NaN,NaN,NaN,NaN,NaN,NaN
less: Taxes,NaN,NaN,NaN,NaN,NaN,NaN
Net Income,NaN,NaN,NaN,NaN,NaN,NaN


In [89]:
# Substract D&A to get EBIT
IS.loc['less: D&A'] = IS_assumptions['Depre % of Sales'] * -IS.loc['Sales']
IS

,1,2,3,4,5,6
Sales,150,165.0,181.5,199.65,219.615,241.5765
EBITDA,45.0,49.5,54.45,59.895,65.8845,72.47295
less: D&A,-7.5,-8.25,-9.075,-9.9825,-10.98075,-12.078825
EBIT,NaN,NaN,NaN,NaN,NaN,NaN
less: Interest,NaN,NaN,NaN,NaN,NaN,NaN
Pre-Tax Income,NaN,NaN,NaN,NaN,NaN,NaN
less: Taxes,NaN,NaN,NaN,NaN,NaN,NaN
Net Income,NaN,NaN,NaN,NaN,NaN,NaN


In [90]:
# Derived EBIT for each year
IS.loc['EBIT'] = IS.loc['EBITDA'] + IS.loc['less: D&A']
IS

,1,2,3,4,5,6
Sales,150,165.0,181.5,199.65,219.615,241.5765
EBITDA,45.0,49.5,54.45,59.895,65.8845,72.47295
less: D&A,-7.5,-8.25,-9.075,-9.9825,-10.98075,-12.078825
EBIT,37.5,41.25,45.375,49.9125,54.90375,60.394125
less: Interest,NaN,NaN,NaN,NaN,NaN,NaN
Pre-Tax Income,NaN,NaN,NaN,NaN,NaN,NaN
less: Taxes,NaN,NaN,NaN,NaN,NaN,NaN
Net Income,NaN,NaN,NaN,NaN,NaN,NaN


In [91]:
# Calculate the debt balances for interest
interest = {
    'sr_bank_debt_interest':.05,
    'sellers_notes_interest':.08,
    'shareholders_loan_interest':.12,
}
interest['senior_bank_debt'] = sources_sche['sen_bank_debt'] * interest['sr_bank_debt_interest']
interest['sellers_notes'] = sources_sche['sellers_notes'] * interest['sellers_notes_interest']
interest['shareholders_loan'] = equity_split['shareholder_loan'] * interest['shareholders_loan_interest']
interest

{'sr_bank_debt_interest': 0.05,
 'sellers_notes_interest': 0.08,
 'shareholders_loan_interest': 0.12,
 'senior_bank_debt': 6.75,
 'sellers_notes': 3.6,
 'shareholders_loan': 5.3999999999999995}

In [92]:
# Substract interest to get pre-tax income
IS.loc['less: Interest']= -(interest['senior_bank_debt'] + interest['sellers_notes'] + interest['shareholders_loan'])
IS

,1,2,3,4,5,6
Sales,150,165.0,181.5,199.65,219.615,241.5765
EBITDA,45.0,49.5,54.45,59.895,65.8845,72.47295
less: D&A,-7.5,-8.25,-9.075,-9.9825,-10.98075,-12.078825
EBIT,37.5,41.25,45.375,49.9125,54.90375,60.394125
less: Interest,-15.75,-15.75,-15.75,-15.75,-15.75,-15.75
Pre-Tax Income,NaN,NaN,NaN,NaN,NaN,NaN
less: Taxes,NaN,NaN,NaN,NaN,NaN,NaN
Net Income,NaN,NaN,NaN,NaN,NaN,NaN


In [93]:
# Calculate the Pre-Tax Income
IS.loc['Pre-Tax Income'] = IS.loc['EBIT'] + IS.loc['less: Interest']
IS

,1,2,3,4,5,6
Sales,150,165.0,181.5,199.65,219.615,241.5765
EBITDA,45.0,49.5,54.45,59.895,65.8845,72.47295
less: D&A,-7.5,-8.25,-9.075,-9.9825,-10.98075,-12.078825
EBIT,37.5,41.25,45.375,49.9125,54.90375,60.394125
less: Interest,-15.75,-15.75,-15.75,-15.75,-15.75,-15.75
Pre-Tax Income,21.75,25.5,29.625,34.1625,39.15375,44.644125
less: Taxes,NaN,NaN,NaN,NaN,NaN,NaN
Net Income,NaN,NaN,NaN,NaN,NaN,NaN


In [94]:
# Calculate the tax rate 
IS.loc['less: Taxes'] = IS.loc['Pre-Tax Income'] * -IS_assumptions['tax_rate']
IS

,1,2,3,4,5,6
Sales,150,165.0,181.5,199.65,219.615,241.5765
EBITDA,45.0,49.5,54.45,59.895,65.8845,72.47295
less: D&A,-7.5,-8.25,-9.075,-9.9825,-10.98075,-12.078825
EBIT,37.5,41.25,45.375,49.9125,54.90375,60.394125
less: Interest,-15.75,-15.75,-15.75,-15.75,-15.75,-15.75
Pre-Tax Income,21.75,25.5,29.625,34.1625,39.15375,44.644125
less: Taxes,-6.525,-7.65,-8.8875,-10.24875,-11.746125,-13.393238
Net Income,NaN,NaN,NaN,NaN,NaN,NaN


In [95]:
# Calculate the NI
IS.loc['Net Income'] = IS.loc['Pre-Tax Income'] + IS.loc['less: Taxes']
IS

,1,2,3,4,5,6
Sales,150,165.0,181.5,199.65,219.615,241.5765
EBITDA,45.0,49.5,54.45,59.895,65.8845,72.47295
less: D&A,-7.5,-8.25,-9.075,-9.9825,-10.98075,-12.078825
EBIT,37.5,41.25,45.375,49.9125,54.90375,60.394125
less: Interest,-15.75,-15.75,-15.75,-15.75,-15.75,-15.75
Pre-Tax Income,21.75,25.5,29.625,34.1625,39.15375,44.644125
less: Taxes,-6.525,-7.65,-8.8875,-10.24875,-11.746125,-13.393238
Net Income,15.225,17.85,20.7375,23.91375,27.407625,31.250888


In [105]:
# Rename the columns
IS = IS.rename(columns={1: 'Year 1', 2: 'Year 2', 3: 'Year 3', 4: 'Year 4', 5: 'Year 5', 6: 'Year 6' })

# Round the df to only 2 dec. places
decimals = 2    
IS['Year 3'] = IS['Year 3'].apply(lambda x: round(x, decimals))
IS['Year 4'] = IS['Year 4'].apply(lambda x: round(x, decimals))
IS['Year 5'] = IS['Year 5'].apply(lambda x: round(x, decimals))
IS['Year 6'] = IS['Year 6'].apply(lambda x: round(x, decimals))
IS

,Year 1,Year 2,Year 3,Year 4,Year 5,Year 6
Sales,150,165.0,181.50,199.65,219.62,241.58
EBITDA,45.0,49.5,54.45,59.90,65.88,72.47
less: D&A,-7.5,-8.25,-9.08,-9.98,-10.98,-12.08
EBIT,37.5,41.25,45.38,49.91,54.90,60.39
less: Interest,-15.75,-15.75,-15.75,-15.75,-15.75,-15.75
Pre-Tax Income,21.75,25.5,29.63,34.16,39.15,44.64
less: Taxes,-6.525,-7.65,-8.89,-10.25,-11.75,-13.39
Net Income,15.225,17.85,20.74,23.91,27.41,31.25


### V. Cumulative Levered Free Cash Flow

In [97]:
# Create a free cash flow df
FCF = pd.DataFrame(data = None, 
                   index = ['Net Income', 'plus: D&A', 'plus: non-cash interest', 
                                         'less: change in NWC', 'less: CapEx', 'FCF'], 
                   columns = years)
FCF

,1,2,3,4,5,6
Net Income,NaN,NaN,NaN,NaN,NaN,NaN
plus: D&A,NaN,NaN,NaN,NaN,NaN,NaN
plus: non-cash interest,NaN,NaN,NaN,NaN,NaN,NaN
less: change in NWC,NaN,NaN,NaN,NaN,NaN,NaN
less: CapEx,NaN,NaN,NaN,NaN,NaN,NaN
FCF,NaN,NaN,NaN,NaN,NaN,NaN


In [98]:
# Calculate the FCF 
FCF.loc['Net Income'] = IS.loc['Net Income']
FCF.loc['plus: D&A'] = -IS.loc['less: D&A']
FCF.loc['plus: non-cash interest'] = -IS.loc['less: Interest'] - interest['senior_bank_debt']
FCF.loc['less: change in NWC'] = -IS_assumptions['change_in_NWC']
FCF.loc['less: CapEx'] = IS.loc['Sales'] * -IS_assumptions['CAPEX % of Sales']
FCF.loc['FCF'] = FCF.loc[['Net Income', 'plus: D&A', 'plus: non-cash interest', 
                                         'less: change in NWC', 'less: CapEx']].sum()
FCF

,1,2,3,4,5,6
Net Income,15.225,17.85,20.7375,23.91375,27.407625,31.250888
plus: D&A,7.5,8.25,9.075,9.9825,10.98075,12.078825
plus: non-cash interest,9.0,9.0,9.0,9.0,9.0,9.0
less: change in NWC,-3,-3,-3,-3,-3,-3
less: CapEx,-10.5,-11.55,-12.705,-13.9755,-15.37305,-16.910355
FCF,18.225,20.55,23.1075,25.92075,29.015325,32.419358


In [99]:
# Calculate the FCF 
cumulative_fcf = FCF[[1,2,3,4,5]].loc['FCF'].sum()
cumulative_fcf

116.81857500000002

### VI. Exit Value and Returns

In [100]:
# Create the exit calculations table
exit_equity_value = {
    'exit_multiple':10,
}
exit_equity_value['Exit_EV'] = exit_equity_value['exit_multiple'] * IS[6].loc['EBITDA']
exit_equity_value['Debt'] = sources_sche['sen_bank_debt'] - sources_sche['sellers_notes']
exit_equity_value['Exit_Equity_Value'] = exit_equity_value['Exit_EV'] + exit_equity_value['Debt']
exit_equity_value

{'exit_multiple': 10,
 'Exit_EV': 724.7295000000003,
 'Debt': 90.0,
 'Exit_Equity_Value': 814.7295000000003}

In [101]:
# Create the returns attribution table
returns_attribution = {
    'exit_multiple':10,
}
returns_attribution['EBITDA_Growth'] = (IS[6].loc['EBITDA'] - IS[1].loc['EBITDA']) * entry_assumptions['purchase_multiple']
returns_attribution['Multiple_Expansion'] = returns_attribution['exit_multiple'] * IS[6]['EBITDA'] 
returns_attribution['Returns_to_Equity'] = (exit_equity_value['Exit_EV'] - exit_equity_value['Debt'])
returns_attribution['Debt_Paydown'] = returns_attribution['Returns_to_Equity'] - (returns_attribution['EBITDA_Growth'] + returns_attribution['Multiple_Expansion'])
returns_attribution

{'exit_multiple': 10,
 'EBITDA_Growth': 274.72950000000026,
 'Multiple_Expansion': 724.7295000000003,
 'Returns_to_Equity': 634.7295000000003,
 'Debt_Paydown': -364.72950000000026}

In [102]:
# Create a dict to save variable for MoM
MoM = {
}

MoM['money_on_money_multiple'] = exit_equity_value['Exit_Equity_Value'] / sources_sche['equity_funding']
MoM

{'money_on_money_multiple': 2.9201774193548395}

In [103]:
# Create a dict to save variable for IRR
IRR = {
}

IRR['internal_rate_of_return'] = (exit_equity_value['Exit_Equity_Value'] / sources_sche['equity_funding']) ** (1/5) - 1
IRR

{'internal_rate_of_return': 0.23903007368331464}